# Criação do modelo de Machine Learning, capaz de fazer a verificação dos sentimentos de cada frase

In [1]:
# Imports
import numpy as np
import pandas as pd
from unidecode import unidecode
import spacy

import nltk
nltk.download('stopwords')
nltk.download('rslp')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alyss\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\alyss\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier

In [4]:
data = pd.read_csv("C:/Projetos Pessoais/dados/imdb-reviews-pt-br.csv")

In [5]:
data.head()

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg


In [6]:
# Info do conjunto de dados
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49459 entries, 0 to 49458
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         49459 non-null  int64 
 1   text_en    49459 non-null  object
 2   text_pt    49459 non-null  object
 3   sentiment  49459 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.5+ MB


In [7]:
data['sentiment_int'] = data['sentiment'].map({"pos":1, "neg":0})

In [8]:
data.sample()

,id,text_en,text_pt,sentiment,sentiment_int
10446,10448,This movie was so awful that I cant even descr...,Este filme foi tão horrível que eu não consigo...,neg,0


In [14]:
# Load dados em portugues

nlp = spacy.load("pt_core_news_lg")

OSError: [E050] Can't find model 'pt_core_news_lg'. It doesn't seem to be a Python package or a valid path to a data directory.

### <font color="red"> Criando função de Lematização para os verbos

In [15]:
def lematizer(text):

   # Lista de sentenças e documentos
   sentencas = []
   doc = nlp(text) # leitura do text e interpretar em lg de processamento

   for palavra in doc:
      if palavra.pos_ == "VERB":
         sentencas.append(palavra.lemma_) # se for verbo, extrai o lemma
      else:
         sentencas.append(palavra.orth_) # se n for verbo, extrai a palavra

   return " ".join(sentencas)

In [14]:
data['text'] = data['text_pt'].apply(str.lower)
data['text'] = data['text'].apply(unidecode)
data['text'] = data['text'].apply(lematizer)

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49459 entries, 0 to 49458
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             49459 non-null  int64 
 1   text_en        49459 non-null  object
 2   text_pt        49459 non-null  object
 3   sentiment      49459 non-null  object
 4   sentiment_int  49459 non-null  int64 
 5   text           49459 non-null  object
dtypes: int64(2), object(4)
memory usage: 2.3+ MB


In [17]:
# Salvando novo DF processado - Endpoint
data.to_csv("C:/Projetos Pessoais/dados/imdb_dados_processados.csv")

In [9]:
data = pd.read_csv("C:/Projetos Pessoais/dados/imdb_dados_processados.csv")

In [10]:
df = data.copy()

### <font color="red"> Tratamento de Stopwords

In [11]:
stop_words = nltk.corpus.stopwords.words('portuguese')

In [12]:
stop_words[:8]

['a', 'à', 'ao', 'aos', 'aquela', 'aquelas', 'aquele', 'aqueles']

### <font color="red"> Contagem de Unigramas

In [13]:
vect_uni_cv = CountVectorizer(ngram_range=(1,1), stop_words = stop_words)
vect_uni_cv.fit(df.text)
text_vect_uni_cv = vect_uni_cv.transform(df.text)

In [14]:
x_train, x_test, y_train, y_test = train_test_split(text_vect_uni_cv,
                                                    df["sentiment_int"],
                                                    test_size = 0.2,
                                                    random_state = 321)

In [15]:
x_train

<39567x114566 sparse matrix of type '<class 'numpy.int64'>'
	with 4081905 stored elements in Compressed Sparse Row format>

### <font color="red"> Modelo v1 => Regressão Logística

In [16]:
# Logistic Regression

lr = LogisticRegression()
lr.fit(x_train, y_train)

LogisticRegression()

In [17]:
lr_prev_modelo_1 = lr.predict(x_test)

# Acurácia
acc_v1 = accuracy_score(lr_prev_modelo_1, y_test)
acc_v1

0.8813182369591589

In [20]:
import pickle

modelo_v1_metricas = {
 "Modelo": "Logistic Regression",
 "Versão": "1",
 "Detalhes": " - ",
 "F1_Score": round(f1_score(lr_prev_modelo_1, y_test), 3),
 "Acurácia": round(accuracy_score(lr_prev_modelo_1, y_test), 3),
 "AUC": round(roc_auc_score(y_test, lr_prev_modelo_1), 3) #AUC inverte a ordem dos dados -  primeiro real depois previsões
}

with open("./models/modelo_v1.pkl", "wb") as file:  # "wb" significa escrita em modo binário
    pickle.dump(lr_prev_modelo_1, file)

modelo_v1_metricas

{'Modelo': 'Logistic Regression',
 'Versão': '1',
 'Detalhes': ' - ',
 'F1_Score': 0.882,
 'Acurácia': 0.881,
 'AUC': 0.881}

### <font color="red">Modelo v2 => Regressão Logística com Grid Search

In [21]:
lr_grid = LogisticRegression()
lr_grid_values = {
   'penalty': ['l1','l2'],
   'C': [0.001, 0.009, .09, 1, 5, 10, 25]
}

In [22]:
lr_grid_cv = GridSearchCV(lr_grid,
                          param_grid = lr_grid_values,
                          scoring = 'accuracy',
                          cv=5)

In [23]:
lr_grid_cv.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [0.001, 0.009, 0.09, 1, 5, 10, 25],
                         'penalty': ['l1', 'l2']},
             scoring='accuracy')

In [24]:
lr_grid_cv.best_estimator_

LogisticRegression(C=0.09)

In [25]:
y_predict_lr_GS = lr_grid_cv.predict(x_test)

acc_v2 = accuracy_score(y_predict_lr_GS, y_test)
acc_v2

0.8867771936918722

In [29]:
modelo_v2_metricas = {
 "Modelo": "Logistic Regression GS",
 "Versão": "2",
 "Detalhes": "C=09",
 "F1_Score": round(f1_score(y_predict_lr_GS, y_test), 3),
 "Acurácia": round(accuracy_score(y_predict_lr_GS, y_test), 3),
 "AUC": round(roc_auc_score(y_test, y_predict_lr_GS), 3) #AUC inverte a ordem dos dados -  primeiro real depois previsões
}

with open("./models/modelo_v2.pkl", "wb") as file:  # "wb" significa escrita em modo binário
    pickle.dump(lr_grid_cv, file)
modelo_v2_metricas

{'Modelo': 'Logistic Regression GS',
 'Versão': '2',
 'Detalhes': 'C=09',
 'F1_Score': 0.888,
 'Acurácia': 0.887,
 'AUC': 0.887}

### <font color="red">Modelo v3 => Multinomial Naive Bayes

In [27]:
naive_bayes = MultinomialNB()

param_grid_nb = {
    'alpha': [0.01, 0.001, 0.1, 1, 10],  # Regularização
    'fit_prior': [True, False]     # Se as probabilidades aprioristas devem ser ajustadas
}

naive_Bayes_model_v3 = GridSearchCV(naive_bayes,
                          param_grid = param_grid_nb,
                          cv=5,
                          n_jobs=-1)

naive_Bayes_model_v3.fit(x_train, y_train)

naive_Bayes_model_v3.best_estimator_

MultinomialNB(alpha=1, fit_prior=False)

In [28]:
naive_bayes_predict_v3 = naive_Bayes_model_v3.predict(x_test)

In [30]:
modelo_v3_metricas = {
 "Modelo": "Multinomial Naive Bayes",
 "Versão": "3",
 "Detalhes": "alfa=1",
 "F1_Score": round(f1_score(naive_bayes_predict_v3, y_test), 3),
 "Acurácia": round(accuracy_score(naive_bayes_predict_v3, y_test), 3),
 "AUC": round(roc_auc_score(y_test, naive_bayes_predict_v3), 3) #AUC inverte a ordem dos dados -  primeiro real depois previsões
}

with open("./models/modelo_v3.pkl", "wb") as file:  # "wb" significa escrita em modo binário
    pickle.dump(naive_Bayes_model_v3, file)


modelo_v3_metricas

{'Modelo': 'Multinomial Naive Bayes',
 'Versão': '3',
 'Detalhes': 'alfa=1',
 'F1_Score': 0.856,
 'Acurácia': 0.859,
 'AUC': 0.859}

### <font color="red">Modelo v4 => Decision Tree com Grid Search

In [ ]:
clf_decision_tree = DecisionTreeClassifier()

params_decision_tree = {
 "criterion": ["gini", "entropy"],
 "max_depth": [3, None],
 "min_samples_leaf": [1, 3, 10],
 "min_samples_split": [2, 3],
 "max_depth": [1,3,10]
}

modelo_v2_decisionTree = GridSearchCV(clf_decision_tree,
                                      params_decision_tree,
                                      scoring='accuracy',
                                      cv = 5)

modelo_v2_decisionTree.fit(x_train, y_train)

### <font color="red">Modelo v7 => Random Forest Classifier

In [75]:
clf_random_forest = RandomForestClassifier(n_estimators = 100)

params_random_forest = {
    'max_depth': [None, 2],  # Profundidade máxima de cada árvore
    'min_samples_split': [2, 5],  # Mínimo de amostras necessárias para dividir um nó
    'min_samples_leaf': [1, 2, 3],    # Mínimo de amostras necessárias em um nó folha
    'criterion': ['gini', 'entropy']
}

modelo_v4_randomForest = GridSearchCV(clf_random_forest,
                                      params_random_forest,
                                      cv = 5)

modelo_v4_randomForest.fit(x_train, y_train)

modelo_v4_randomForest.best_params_

KeyboardInterrupt: 

In [ ]:
predict_modelo_v4_randomForest = modelo_v4_randomForest.predict(x_test)

modelo_v4_metricas = {
 "Modelo": "Random Forest",
 "Versão": "4",
 "Detalhes": " ",
 "F1_Score": round(f1_score(predict_modelo_v4_randomForest, y_test), 3),
 "Acurácia": round(accuracy_score(predict_modelo_v4_randomForest, y_test), 3),
 "AUC": round(roc_auc_score(y_test, predict_modelo_v4_randomForest), 3) #AUC inverte a ordem dos dados -  primeiro real depois previsões
}
modelo_v4_metricas

### <font color="red">Modelo v5 => SVC

In [78]:
clf_svc = SVC()

param = {
 'kernel' : ['Linear', 'rbf', 'poly']
}

modelo_v7_SVC_GS = GridSearchCV(clf_svc, param, cv = 3) # cv

modelo_v7_SVC_GS.fit(x_train, y_train)


In [1]:
predict_modelo_v5_SVC = modelo_v7_SVC_GS.predict(x_test)

modelo_v5_metricas = {
 "Modelo": "Random Forest",
 "Versão": "4",
 "Detalhes": " ",
 "F1_Score": round(f1_score(predict_modelo_v5_SVC, y_test), 3),
 "Acurácia": round(accuracy_score(predict_modelo_v5_SVC, y_test), 3),
 "AUC": round(roc_auc_score(y_test, predict_modelo_v5_SVC), 3) #AUC inverte a ordem dos dados -  primeiro real depois previsões
}
modelo_v5_metricas

NameError: name 'modelo_v7_SVC_GS' is not defined

### <font color="red">Modelo v6 => Adaboost